In [55]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from tensorflow import keras as keras
from matplotlib import pyplot as plt

from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Activation, Input, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, Reshape, Concatenate, concatenate

### 数据处理

In [56]:
# 读取数据
data_ini = pd.read_excel('data/TJ POLTS.xlsx')

data = data_ini.iloc[:,[1,3,4,5,6,7,8]]
data.head()

,AQI,PM2.5,PM10,NO2,SO2,CO,O3_8h
0,310,260,420,139,201,3.5,13
1,225,175,271,111,143,3.2,31
2,275,225,360,128,197,3.9,12
3,126,96,183,70,101,2.3,26
4,116,88,159,70,114,1.6,16


In [57]:
# 指数平滑
def expPreprocessing(df, alpha=0.05):
    edata = df.ewm(alpha=alpha, adjust=False).mean()
    return edata

alpha = 0.05
data_exp = expPreprocessing(data, alpha)
data_exp.head()

# alpha = 0.05
# data_exp = data.ewm(alpha=alpha, adjust=False).mean()
# data_exp.head()

,AQI,PM2.5,PM10,NO2,SO2,CO,O3_8h
0,310.000000,260.000000,420.000000,139.0000,201.000000,3.500000,13.000000
1,305.750000,255.750000,412.550000,137.6000,198.100000,3.485000,13.900000
2,304.212500,254.212500,409.922500,137.1200,198.045000,3.505750,13.805000
3,295.301875,246.301875,398.576375,133.7640,193.192750,3.445462,14.414750
4,286.336781,238.386781,386.597556,130.5758,189.233112,3.353189,14.494013


In [58]:
# 归一化
min_value = data_exp.min(axis=0)  
max_value = data_exp.max(axis=0)

data_std = (data_exp - min_value) / (max_value - min_value)

In [59]:
# 划分训练集与验证集

time_stamp = 50  # 时间点长度
ratio = 0.8

split = int(ratio*len(data))
train_data = data_std[0:split + time_stamp]
valid_data = data_std[split - time_stamp:]

In [60]:
label_column = 0 # 标签所在的列id
forecast_horizon = 5 # 预测的步数
x_train, y_train = [], []
scaled_data = train_data.values
# 训练集
for i in range(time_stamp, len(train_data) - forecast_horizon + 1):
    x_train.append(scaled_data[i - time_stamp:i])
    y_train.append(scaled_data[i:i + forecast_horizon, label_column])

x_train, y_train = np.array(x_train), np.array(y_train)

# 验证集
x_valid, y_valid = [], []
for i in range(time_stamp, len(valid_data) - forecast_horizon + 1):
    x_valid.append(scaled_data[i - time_stamp:i])
    y_valid.append(scaled_data[i:i + forecast_horizon, label_column])

x_valid, y_valid = np.array(x_valid), np.array(y_valid)


### 网络定义

In [61]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

def ADNNet_model(x, hidden_1, hidden_2, dropout_rate=0.1, attention_dim=11, num_heads=4):
    x_input = Input(shape=(x.shape[1], x.shape[2]))
    # x_input1 = layers.Conv1D(filters=34, kernel_size=1, padding='same', activation='relu')(x_input)
    # x_input1 = layers.MaxPool1D(pool_size=7, padding='valid')(x_input1)
    
    AQI, P = x_input[:,:,0], x_input[:,:,1:]
    feature_time_lag = mlp(AQI, [hidden_1, hidden_2], dropout_rate)
    feature_pol_con = mlp(P, [hidden_1, hidden_2], dropout_rate)
    feature_seasonal = mlp(x_input, [hidden_1, hidden_2], dropout_rate)
    
    x_input1= layers.Dense(hidden_2)(x_input)
    '''
    ######################### 方法一 ######################### 
    x_features = layers.Add()([x_input1, feature_time_lag, feature_pol_con, feature_seasonal])
    
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=attention_dim, dropout=0.0799
    )(x_input1, x_features)
    # Skip connection 1.
    x2 = layers.Add()([attention_output, x_input1])
    '''
    '''
    ######################### 方法二 ######################### 
    x_features = layers.Add()([x_input1, feature_time_lag, feature_pol_con, feature_seasonal])
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=attention_dim, dropout=0.0799
    )(x_features, x_features)
    # Skip connection 1.
    x2 = layers.Add()([attention_output, x_features])
    '''
    ######################### 方法三 ######################### 
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=attention_dim, dropout=0.0799
    )(x_input1, x_input1)
    # Skip connection 1.
    x2 = layers.Add()([attention_output, feature_time_lag, feature_pol_con, feature_seasonal])
    
    
    # Layer normalization 2.
    x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
    # MLP.
    x3 = mlp(x3, hidden_units=[hidden_2], dropout_rate=dropout_rate)

    # Skip connection 2.
    encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, attention_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(dropout_rate)(representation)
    # Classify outputs.
    logits = layers.Dense(5)(representation)
    logits = layers.Dropout(dropout_rate)(logits)
    # Create the Keras model.
    return Model(inputs=x_input, outputs=logits)

### 贝叶斯参数优化

In [62]:
import optuna

class Trainer():
    def __init__(self, 
                 x_train, 
                 y_train, 
                 x_valid, 
                 y_valid,  
                 num_heads=4,
                 epochs=50, 
                 batch_size=64,
                 n_trials=100
                ):
        self.x_train, self.y_train, self.x_valid, self.y_valid = x_train, y_train, x_valid, y_valid
        self.num_heads = num_heads
        self.epochs = epochs
        self.batch_size = batch_size
        self.n_trials = n_trials
        

    def objective(self, trial):
        lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
        hidden_1 = trial.suggest_int('hidden_1', 32, 256)
        hidden_2 = trial.suggest_int('hidden_2', 32, 256)
        dropout_rate = trial.suggest_uniform('dropout_rate', 0., 0.01)
        attention_dim = trial.suggest_int('attention_dim', 8, 128)
        
        model = ADNNet_model(x=self.x_train, 
                                  hidden_1=hidden_1, 
                                  hidden_2=hidden_2, 
                                  dropout_rate=dropout_rate, 
                                  attention_dim=attention_dim, 
                                  num_heads=self.num_heads
                                 )
        optimizer = keras.optimizers.Adam(learning_rate=lr)
        model.compile(loss='mean_squared_error', optimizer=optimizer)
        model.fit(self.x_train, self.y_train, epochs=self.epochs, batch_size=self.batch_size, verbose=0)

        #score = model.evaluate(x_valid, y_valid, verbose=0)
        y_pred = model.predict(self.x_valid)
        score = mean_squared_error(self.y_valid, y_pred)

        return score
    
    def optimizer_optuna(self):
        algo = optuna.samplers.TPESampler()
        study = optuna.create_study(sampler=algo, direction='minimize')
        study.optimize(self.objective, n_trials=self.n_trials, show_progress_bar=True)
        return study

In [63]:
# 超参数
epochs = 5
batch_size = 64
num_heads = 4
n_trials = 10

T = Trainer(x_train, y_train, x_valid, y_valid, num_heads=num_heads, epochs=epochs, batch_size=batch_size, n_trials=n_trials)
study = T.optimizer_optuna()
print(study.best_params)
print(study.best_value)

[I 2024-01-30 10:14:35,724] A new study created in memory with name: no-name-556fdaf6-3514-42f4-9514-ac4d1c794958
D:\anaconda3\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


[I 2024-01-30 10:14:51,132] Trial 0 finished with value: 0.008084374459970088 and parameters: {'lr': 0.0022544666868174884, 'hidden_1': 239, 'hidden_2': 175, 'dropout_rate': 0.009001480814138383, 'attention_dim': 31}. Best is trial 0 with value: 0.008084374459970088.
[I 2024-01-30 10:15:10,505] Trial 1 finished with value: 0.0045402224182778325 and parameters: {'lr': 0.003043909769740776, 'hidden_1': 125, 'hidden_2': 249, 'dropout_rate': 0.0055252819649870616, 'attention_dim': 48}. Best is trial 1 with value: 0.0045402224182778325.
[I 2024-01-30 10:15:30,268] Trial 2 finished with value: 0.002704850136783534 and parameters: {'lr': 0.00037361014766595344, 'hidden_1': 227, 'hidden_2': 155, 'dropout_rate': 0.00048369551964423717, 'attention_dim': 100}. Best is trial 2 with value: 0.002704850136783534.
[I 2024-01-30 10:15:43,772] Trial 3 finished with value: 0.0032983334382383532 and parameters: {'lr': 0.0004532588106085918, 'hidden_1': 185, 'hidden_2': 139, 'dropout_rate': 0.0015756014750

In [64]:
# 根据最优超参，训练和预测
params = study.best_params
lr = params['lr']
hidden_1 = params['hidden_1']
hidden_2 = params['hidden_2']
dropout_rate = params['dropout_rate']
attention_dim = params['attention_dim']

checkpoint_path_best = "data/best.hdf5"
modelcheckpoint_best = keras.callbacks.ModelCheckpoint(checkpoint_path_best,
                                                       monitor='loss',
                                                       save_best_only=True, 
                                                       mode='min', 
                                                       verbose=0)
model = transformer_model(x=x_train, 
                          hidden_1=hidden_1, 
                          hidden_2=hidden_2, 
                          dropout_rate=dropout_rate, 
                          attention_dim=attention_dim,
                         )
#print(model.summary())
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(learning_rate=lr))
model.fit(x_train, y_train, epochs=50, batch_size=batch_size, verbose=1, callbacks=[modelcheckpoint_best])

model.load_weights(checkpoint_path_best)
closing_price = model.predict(x_valid)

Epoch 1/50
41/41 [==============================] - 5s 100ms/step - loss: 0.5655
Epoch 2/50
 1/41 [..............................] - ETA: 3s - loss: 0.0234

D:\anaconda3\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


41/41 [==============================] - 4s 100ms/step - loss: 0.0101
Epoch 3/50
41/41 [==============================] - 4s 102ms/step - loss: 0.0055
Epoch 4/50
41/41 [==============================] - 4s 101ms/step - loss: 0.0046
Epoch 5/50
41/41 [==============================] - 4s 102ms/step - loss: 0.0045
Epoch 6/50
41/41 [==============================] - 4s 105ms/step - loss: 0.0044
Epoch 7/50
41/41 [==============================] - 4s 103ms/step - loss: 0.0041
Epoch 8/50
41/41 [==============================] - 4s 105ms/step - loss: 0.0043
Epoch 9/50
41/41 [==============================] - 4s 103ms/step - loss: 0.0040
Epoch 10/50
41/41 [==============================] - 4s 103ms/step - loss: 0.0039
Epoch 11/50
41/41 [==============================] - 4s 104ms/step - loss: 0.0041
Epoch 12/50
41/41 [==============================] - 4s 102ms/step - loss: 0.0045
Epoch 13/50
41/41 [==============================] - 4s 104ms/step - loss: 0.0041
Epoch 14/50
41/41 [================

In [65]:
closing_price

array([[0.41351092, 0.34056708, 0.49436292, 0.4236973 , 0.36763814],
       [0.4044084 , 0.32993904, 0.4990879 , 0.41560453, 0.3611808 ],
       [0.4152282 , 0.33575004, 0.48480025, 0.40048563, 0.3471617 ],
       ...,
       [0.11736793, 0.05283482, 0.20053434, 0.10354332, 0.10722256],
       [0.13131185, 0.0553672 , 0.20040154, 0.11721755, 0.10457492],
       [0.13084204, 0.08349486, 0.21330151, 0.14712508, 0.12259388]],
      dtype=float32)

In [66]:
import numpy as np
import pandas as pd

# 假设 max_value, min_value, alpha, label_column 已正确定义
# max_value = {...}
# min_value = {...}
# alpha = ...
# label_column = ...
y_valid_list = [y_valid]
# 反归一化
def unnormalize(data, max_value, min_value, label_column):
    return (data * (max_value[label_column] - min_value[label_column])) + min_value[label_column]

y_valid_original = unnormalize(pd.DataFrame(y_valid_list[0]), max_value, min_value, label_column)
predictions_original = unnormalize(pd.DataFrame(closing_price), max_value, min_value, label_column)

# 反平滑
# def exp_reversed(df, alpha):
#     reversed_df = pd.DataFrame()
#     for column in df.columns:
#         df_col = df[column]
#         row_0 = df_col.iloc[0]
#         df_t_1 = pd.Series([row_0]).append(df_col[:-1]).reset_index(drop=True)
#         result = (df_col - (1 - alpha) * df_t_1) / alpha
#         result.iloc[0] = df_col.iloc[0]
#         reversed_df = pd.concat([reversed_df, result], axis=1)
#     return reversed_df
original_y_valid = y_valid_original
original_predictions = predictions_original
# original_y_valid = exp_reversed(y_valid_original, alpha)
# original_predictions = exp_reversed(predictions_original, alpha)
print(original_y_valid)
print(original_predictions)
# 评估每一步的预测
metrics = {'RMSE': [], 'MAE': [], 'R2Score': [], 'MAPE': []}
for step in range(5):
    # 计算每个指标
    rmse = np.sqrt(mean_squared_error(original_y_valid.iloc[:, step], original_predictions.iloc[:, step]))
    mae = mean_absolute_error(original_y_valid.iloc[:, step], original_predictions.iloc[:, step])
    r2 = r2_score(original_y_valid.iloc[:, step], original_predictions.iloc[:, step])
    mape = np.mean(np.abs((original_y_valid.iloc[:, step] - original_predictions.iloc[:, step]) / original_y_valid.iloc[:, step])) * 100

    # 存储指标
    metrics['RMSE'].append(rmse)
    metrics['MAE'].append(mae)
    metrics['R2Score'].append(r2)
    metrics['MAPE'].append(mape)

# 输出指标
print(metrics)


              0           1           2           3           4
0    167.431957  166.060360  165.857342  169.814475  165.873751
1    166.060360  165.857342  169.814475  165.873751  160.430063
2    165.857342  169.814475  165.873751  160.430063  159.258560
3    169.814475  165.873751  160.430063  159.258560  155.195632
4    165.873751  160.430063  159.258560  155.195632  152.735850
..          ...         ...         ...         ...         ...
646   79.746129   78.708823   79.773382   85.684712   90.000477
647   78.708823   79.773382   85.684712   90.000477   92.700453
648   79.773382   85.684712   90.000477   92.700453   95.865430
649   85.684712   90.000477   92.700453   95.865430   96.222159
650   90.000477   92.700453   95.865430   96.222159   93.211051

[651 rows x 5 columns]
              0           1           2           3           4
0    162.280334  143.907898  182.644623  164.845993  150.726318
1    159.987671  141.231003  183.834717  162.807663  149.099899
2    162.712875 

In [67]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_valid_list = [y_valid]
# # 假设您的模型已经产生了5步预测
# predictions = model.predict(y_valid_list)  # 这应该是一个形状为 (n_samples, 5) 的数组

# 初始化指标
metrics = {'RMSE': [], 'MAE': [], 'R2Score': [], 'MAPE': []}
y_valid = pd.DataFrame(y_valid_list[0])
predictions =  pd.DataFrame(closing_price)
# 对于每一步预测
for step in range(5):
    # 计算每个指标
    rmse = np.sqrt(mean_squared_error(y_valid.iloc[:, step], predictions.iloc[:, step]))
    mae = mean_absolute_error(y_valid.iloc[:, step], predictions.iloc[:, step])
    r2 = r2_score(y_valid.iloc[:, step], predictions.iloc[:, step])
    mape = np.mean(np.abs((y_valid.iloc[:, step] - predictions.iloc[:, step]) / y_valid.iloc[:, step])) * 100

    # 存储指标
    metrics['RMSE'].append(rmse)
    metrics['MAE'].append(mae)
    metrics['R2Score'].append(r2)
    metrics['MAPE'].append(mape)

# 输出指标
print(metrics)


{'RMSE': [0.0306383483519987, 0.050430801259047205, 0.10670781917967968, 0.035227763396814125, 0.03222500289513167], 'MAE': [0.02729811385178442, 0.043631452462003006, 0.10355935255224184, 0.029702633993402384, 0.02394301014189962], 'R2Score': [0.90340607855537, 0.7363717228480356, -0.18893337191901605, 0.8694652790779555, 0.8898513135433184], 'MAPE': [20.10864159798638, 24.258492814531117, 75.93891538016484, 22.993170586387464, 14.14609752123918]}


In [68]:
# # closing_price = scaler.inverse_transform(closing_price)
# # y_valid = scaler.inverse_transform([y_valid])
# y_valid_list = [y_valid]
#
# mse_2 = mean_squared_error(y_valid_list[0], closing_price.reshape(1,-1)[0])
# print("RMSE ", np.sqrt(mse_2))
# mae = mean_absolute_error(y_valid_list[0], closing_price.reshape(1,-1)[0])
# print("MAE ", mae)
# r2 = r2_score(y_valid_list[0], closing_price.reshape(1,-1)[0])
# print("R2SCORE ", r2)
# mape = mean_absolute_percentage_error(y_valid_list[0], closing_price.reshape(1,-1)[0])
# print("MAPE ", mape)

In [69]:

# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
#
# # 反归一化
# closing_price_original = (closing_price.reshape(-1, 1) * (max_value[label_column] - min_value[label_column])) + min_value[label_column]
# y_valid_original = (y_valid.reshape(-1, 1) * (max_value[label_column] - min_value[label_column])) + min_value[label_column]
# # 创建图表
# plt.figure(figsize=(16, 8))
# dict_data = {
#     'Predictions': closing_price_original.flatten(),
#     'Close': y_valid_original.flatten()
# }
# data_pd = pd.DataFrame(dict_data)
# plt.plot(data_pd[['Close', 'Predictions']])
# plt.legend(['Close', 'Predictions'], loc='upper right')
# plt.show()
# # 保存预测值
# pd.DataFrame(closing_price_original).to_csv('data/ADNNet_pred_BJ.csv', index=False)
# pd.DataFrame(y_valid_original).to_csv('data/Real_BJ.csv', index=False)

In [70]:
# # valid.iloc[time_stamp:,0].values == y_valid
#
# train_labels = list(data.iloc[:split,0])   # data.iloc[split:,0]
# test_labels = list(data.iloc[:,0])
#
# prediction_values = closing_price.reshape(1,-1)[0]
# prediction_exp = prediction_values * (max_value[0] - min_value[0]) + min_value[0]
# prediction_list = list(prediction_exp)
# all_labels = train_labels + prediction_list
#
# labels_dict = {'prediction': all_labels, 'test': test_labels}
# df_labels = pd.DataFrame(labels_dict)

In [71]:
# df_labels

In [72]:
# def exp_reversed(df, alpha):
#     row_0 = df.iloc[0]
#     df_t_1 = df.iloc[:0].append(row_0, ignore_index=True).append(df.iloc[:-1], ignore_index=True)
#     results = (df - (1-alpha)*df_t_1)/alpha
#     results.iloc[0] = df.iloc[0]
#     return results
#
# original_labels = exp_reversed(df_labels, alpha)
# original_labels